# Welcome to a brute-force modelling approach of Pair Distribution Function Data

This script first import modules and defines functions.

Afterwards, a parameter range can be set for the structures that is under investigation. In this case we investigate structures up to 200 atoms with 7 different structure types: 

Cubic (sc), body-centered cubic (bcc), face-centered cubic (fcc), hexagonal closed packed (hcp), decahedral, icosahedral and octahedral 

## Import modules and define functions

In [ ]:
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize.minpack import leastsq
from multiprocessing import Pool
from diffpy.Structure import Structure, Atom
from diffpy.srfit.pdf import PDFContribution, DebyePDFGenerator, PDFParser
from diffpy.srfit.fitbase import Profile, FitRecipe, FitContribution, FitResults
from ase.cluster.decahedron import Decahedron
from ase.cluster.icosahedron import Icosahedron
from ase.cluster.octahedron import Octahedron
from ase.cluster.cubic import FaceCenteredCubic, BodyCenteredCubic, SimpleCubic
from ase.lattice.hexagonal import HexagonalClosedPacked
from carbontracker.tracker import CarbonTracker

def fit(Qmin, Qmax, Qdamp, cluster, PDFfile, plot):
    """Function that Fits an experimental PDF using a specific cluster. Set plot=True to get fit parameters."""
    # Create a PDF contribution as before
    pdfprofile = Profile()
    pdfparser = PDFParser()
    pdfparser.parseFile(PDFfile)
    pdfprofile.loadParsedData(pdfparser)
    pdfprofile.setCalculationRange(xmin = 1.5, xmax = 20, dx=0.1)

    # Setup the PDFgenerator that calculates the PDF from the model
    #Generator for first cluster
    pdfgenerator = DebyePDFGenerator("G")
    pdfgenerator.setQmax(Qmax)
    pdfgenerator.setQmin(Qmin)
    pdfgenerator._calc.evaluatortype = 'OPTIMIZED'

    # Input the data files
    pdfgenerator.setStructure(cluster, periodic=False)

    # Add the profile and generator the the PDFcontribution
    pdfcontribution = FitContribution("pdf")
    pdfcontribution.setProfile(pdfprofile, xname="r")
    pdfcontribution.addProfileGenerator(pdfgenerator)
    pdfcontribution.setEquation("scale*G")

    # Moving on
    recipe = FitRecipe()
    recipe.addContribution(pdfcontribution)
    recipe.addVar(pdfcontribution.scale, 0.1, tag = "scale")
    recipe.restrain("scale", lb=0, ub=1e99, sig=0.001)
    pdfgenerator.qdamp.value = Qdamp

    # Add ADP for the cluster
    phase_struc = pdfgenerator.phase
    atoms = phase_struc.getScatterers()

    # Make latice to the phase
    lat = phase_struc.getLattice()
    # Make new variable zoomscale
    recipe.newVar("zoomscale", 1.00, tag = "lat")
    recipe.constrain(lat.a, 'zoomscale')
    recipe.constrain(lat.b, 'zoomscale')
    recipe.constrain(lat.c, 'zoomscale')

    # We create the variables of ADP and assign the initial value to them. In this
    # example, we use isotropic ADP for all atoms
    Biso = recipe.newVar("Biso", value=1, tag = 'ADP')
    recipe.restrain(Biso, lb=0, ub=3, sig = 0.01)
 
    # For all atoms in the structure model, we constrain their Biso according to their species 
    for atom in atoms:
        if atom.element == cluster.element[0]:
            recipe.constrain(atom.Biso, Biso)
        
    recipe.clearFitHooks()
    #     Tune PDF
    recipe.fix("all")
    recipe.free("scale")
    leastsq(recipe.residual, recipe.getValues())   
    recipe.free("ADP")
    leastsq(recipe.residual, recipe.getValues())   
    recipe.free("lat")
    leastsq(recipe.residual, recipe.getValues())   

    # We calculate the goodness-of-fit, Rwp
    g = recipe.pdf.profile.y
    gcalc = recipe.pdf.evaluate()
    rfactor = np.sqrt(sum((g - gcalc)**2) / sum((g)**2))
    
    if plot:
        print ("FIT RESULTS\n")
        res = FitResults(recipe)
        print (res)

        # Plot the observed and refined PDF.
        # Get the experimental data from the recipe
        r = recipe.pdf.profile.x
        gobs = recipe.pdf.profile.y

        # Get the calculated PDF and compute the difference between the calculated and measured PDF
        gcalc = recipe.pdf.evaluate()
        baseline = 1.1 * gobs.min()
        gdiff = gobs - gcalc

        # Plot!
        plt.figure()
        plt.plot(r, gobs, 'bo', label="G(r) data")
        plt.plot(r, gcalc, 'r-', label="G(r) fit")
        plt.plot(r, gdiff + baseline, 'g-', label="G(r) diff")
        plt.plot(r, np.zeros_like(r) + baseline, 'k:')
        plt.xlabel(r"$r (\AA)$")
        plt.ylabel(r"$G (\AA^{-2})$")
        plt.legend()

        plt.show()
    return rfactor
  
def exp_file_parameters(file):
    """Function that takes an gr file as input in PDFGui format and extract:
    Qmin, Qmax, atom, lattice constant and filename."""
    f = open(file, "r")
    file_text = f.read()
    Qmin = float(file_text[file_text.find("qmin")+7:file_text.find("qmin")+10])
    Qmax = float(file_text[file_text.find("qmax ")+7:file_text.find("qmax ")+11])
    atom = file_text[file_text.find("composition")+14:file_text.find("composition")+16]
    try:
        data = np.loadtxt(file)
    except ValueError:
        data = np.loadtxt(file, skiprows=24)
    lc = data[data[:,1].argmax(),0]
    return Qmin, Qmax, atom, lc, file

# Use the brute-force approach on all dataset in the "Data" folder 

In [ ]:
tracker = CarbonTracker(epochs=1, log_dir='./CO2_consumption/')
tracker.epoch_start()

data_path = "Data/"
size_threshold = 200
cores = 1
# Parameter range for SC, BCC and FCC
h_min = 1 #Min 1
h_max = 10
k_min = 1 # Min 1
k_max = 10
l_min = 1 #Min 1
l_max = 10
# Parameter range for HCP
lc1_min = 1 #Min 1
lc1_max = 10
lc2_min = 1 # Min 1
lc2_max = 10
size1_min = 1 #Min 1
size1_max = 10
size2_min = 1 #Min 1
size2_max = 10
size3_min = 1 #Min 1
size3_max = 10
# Parameter range for Icosahedral
shell_min = 1
shell_max = 10
# Parameter range for Decahedral
p_start =1 #min 1
p_end=10
q_start=1 #min 1
q_end=10
r_start=1
r_end=10
# Parameter range for Octahedral
length_min = 1
length_max = 10

# Get all the experimental files
files = sorted(glob.glob(data_path+"*.gr"))
files = files[:8]

def Brute_Force_Metal_Mining(file):
    i = 0
    values = {}
    print ("Brute Force Metal Mining on File: ", file)
    # Determine the exp file parameters 
    Qmin, Qmax, atom, lc, PDFFile = exp_file_parameters(file)
    # Brute Force metal mining of the file
    
    print ("Trying out FCC, BCC and SC")
    # Make FCC, BCC and SC and fit
    for h in range(h_min, h_max):
        for k in range(k_min, k_max):
            for l in range(l_min, l_max):
                surfaces = [(1, 0, 0), (1, 1, 0), (1, 1, 1)]
                layers = [h, k, l]

                # Make FCC and fit
                stru1 = FaceCenteredCubic(atom, surfaces, layers, latticeconstant=2*np.sqrt(0.5*lc**2))
                xyz1 = stru1.get_positions()
                if len(xyz1) < size_threshold:
                    Bi1 = Structure([Atom(atom, xi) for xi in xyz1])
                    rw = fit(Qmin = Qmin, Qmax = Qmax, Qdamp = 0.03, cluster = Bi1, PDFfile=file, plot=False)
                    values[i] = {}
                    values[i]['dataset'] = file.replace(data_path, "")[:-3]
                    values[i]['structuretype'] = 'FCC'
                    values[i]['Rwp'] = rw
                    values[i]['Atoms'] = len(xyz1)
                    values[i]['h'] = h
                    values[i]['k'] = k
                    values[i]['l'] = l
                    i += 1

                # Make BCC and fit
                stru1 = BodyCenteredCubic(atom, surfaces, layers, latticeconstant=lc)
                xyz1 = stru1.get_positions()
                if len(xyz1) < size_threshold:
                    Bi1 = Structure([Atom(atom, xi) for xi in xyz1])
                    rw = fit(Qmin = Qmin, Qmax = Qmax, Qdamp = 0.03, cluster = Bi1, PDFfile=PDFFile, plot=False)
                    values[i] = {}
                    values[i]['dataset'] = file.replace(data_path, "")[:-3]
                    values[i]['structuretype'] = 'BCC'
                    values[i]['Rwp'] = rw
                    values[i]['Atoms'] = len(xyz1)
                    values[i]['h'] = h
                    values[i]['k'] = k
                    values[i]['l'] = l
                    i += 1

                # Make SC and fit
                stru1 = SimpleCubic(atom, surfaces, layers, latticeconstant=lc)
                xyz1 = stru1.get_positions()
                if len(xyz1) < size_threshold:
                    Bi1 = Structure([Atom(atom, xi) for xi in xyz1])
                    rw = fit(Qmin = Qmin, Qmax = Qmax, Qdamp = 0.03, cluster = Bi1, PDFfile=PDFFile, plot=False)
                    values[i] = {}
                    values[i]['dataset'] = file.replace(data_path, "")[:-3]
                    values[i]['structuretype'] = 'SC'
                    values[i]['Rwp'] = rw
                    values[i]['Atoms'] = len(xyz1)
                    values[i]['h'] = h
                    values[i]['k'] = k
                    values[i]['l'] = l
                    i += 1

    print ("Trying out HCP")
    # Make HCP and Fit
    for lc1 in range(lc1_min, lc1_max):
        for lc2 in range(lc2_min, lc2_max):
            for size1 in range(size2_min, size1_max):
                for size2 in range(size2_min, size2_max):
                    for size3 in range(size3_min, size3_max):
                        layers = [h, k, l]
                        stru1 = HexagonalClosedPacked(atom, latticeconstant=(lc1, lc2), size=(size1, size2, size3))
                        xyz1 = stru1.get_positions()
                        if len(xyz1) < size_threshold:
                            Bi1 = Structure([Atom(atom, xi) for xi in xyz1])
                            rw = fit(Qmin = Qmin, Qmax = Qmax, Qdamp = 0.03, cluster = Bi1, PDFfile=PDFFile, plot=False)
                            values[i] = {}
                            values[i]['dataset'] = file.replace(data_path, "")[:-3]
                            values[i]['structuretype'] = 'HCP'
                            values[i]['Rwp'] = rw
                            values[i]['Atoms'] = len(xyz1)
                            values[i]['lc1'] = lc1
                            values[i]['lc2'] = lc2
                            values[i]['size1'] = size1
                            values[i]['size2'] = size2
                            values[i]['size3'] = size3
                            i += 1

    print ("Trying out Icosahedral")
    # Make Icosahedral and Fit
    for shell in range(shell_min,shell_max):
        stru1 = Icosahedron(atom, shell,latticeconstant=lc)
        xyz1 = stru1.get_positions()
        if len(xyz1) < size_threshold:
            Bi1 = Structure([Atom(atom, xi) for xi in xyz1])
            rw = fit(Qmin = Qmin, Qmax = Qmax, Qdamp = 0.03, cluster = Bi1, PDFfile=PDFFile, plot=False)
            values[i] = {}
            values[i]['dataset'] = file.replace(data_path, "")[:-3]
            values[i]['structuretype'] = 'Icosahedral'
            values[i]['Rwp'] = rw
            values[i]['Atoms'] = len(xyz1)
            values[i]['shell'] = shell
            i += 1

    print ("Trying out Decahedral")  
    # Make Decahedral and Fit  
    for p in range(p_start,p_end):
        for q in range(q_start,q_end):
            for r in range(r_start, r_end):
                stru1 = Decahedron(atom, p, q, r,latticeconstant=lc)
                xyz1 = stru1.get_positions()
                if len(xyz1) < size_threshold:
                    Bi1 = Structure([Atom(atom, xi) for xi in xyz1])
                    rw = fit(Qmin = Qmin, Qmax = Qmax, Qdamp = 0.03, cluster = Bi1, PDFfile=PDFFile, plot=False)
                    values[i] = {}
                    values[i]['dataset'] = file.replace(data_path, "")[:-3]
                    values[i]['structuretype'] = 'Decahedral'
                    values[i]['Rwp'] = rw
                    values[i]['Atoms'] = len(xyz1)
                    values[i]['p'] = p
                    values[i]['q'] = q
                    values[i]['r'] = r
                    i += 1

    print ("Trying out Octahedral")
    # Make Octahedral and Fit  
    for length in range(length_min, length_max):
        stru1 = Octahedron(atom, length, latticeconstant=2*np.sqrt(0.5*lc**2))
        xyz1 = stru1.get_positions()
        if len(xyz1) < size_threshold:
            Bi1 = Structure([Atom(atom, xi) for xi in xyz1])
            rw = fit(Qmin = Qmin, Qmax = Qmax, Qdamp = 0.03, cluster = Bi1, PDFfile=PDFFile, plot=False)
            values[i] = {}
            values[i]['dataset'] = file.replace(data_path, "")[:-3]
            values[i]['structuretype'] = 'Octahedral'
            values[i]['Rwp'] = rw
            values[i]['Atoms'] = len(xyz1)
            values[i]['p'] = length
            i += 1


    df = pd.DataFrame(values)
    df.to_csv(save_path+ file.replace(data_path, "")[:-3] + ".csv")
    return None

p = Pool(processes=cores)
results = p.map(Brute_Force_Metal_Mining, files)
p.close()
p.join()
results

tracker.epoch_end()
tracker.stop()
parser.print_aggregate(log_dir="./CO2_consumption/")
